## **TABLAS**
1. proveedores
* proveedor_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* nombre (TEXT)
* direccion (TEXT)
* ciudad (TEXT)
* provincia (TEXT)
* codigo_proveedor (TEXT, UNIQUE)

2. categorias
* categoria_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* nombre (TEXT)
* codigo_categoria (TEXT, UNIQUE)

3. piezas
* pieza_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* codigo_pieza (TEXT, UNIQUE)
* nombre (TEXT)
* color (TEXT)
* precio (FLOAT)
* categoria_id (INT, FOREIGN KEY, REF: categorias(categoria_id))

4. suministros
* suministro_id (INT, PRIMARY KEY, AUTO_INCREMENT)
* proveedor_id (INT, FOREIGN KEY, REF: proveedores(proveedor_id))
* pieza_id (INT, FOREIGN KEY, REF: piezas(pieza_id))
* fecha (DATE)
* cantidad (INT)

## **RELACIONES**

1. Proveedores pueden suministrar varias piezas en diferentes fechas, asi que existe una relacion ***1 to many*** between **proveedores** y **suministros**.

2. Una pieza puede ser suministrada por varios proveedores, asi que existe una relacion ***1 to many*** entre **piezas** y **suministros**.
Un proveedor puede suministrar varias piezas también --> relación **N:M**

3. Cada pieza pertenece a una categoria, asi que existe una relacion ***1 to many*** entre **categorias** y **piezas**.

4. Suministros **N:M**

Tablas de proveedores y categorías quedan en los extremos, suministros y piezas quedan en medio.


In [1]:
import pandas as pd
import sqlite3
from datetime import date

In [2]:
conn = sqlite3.connect('./data/proveedores_piezas.db')
cursor = conn.cursor()

In [3]:
def sqlq(query, conn):
    return pd.read_sql(query, conn)

In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence'")
tables = cursor.fetchall()

for table_name in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name[0]}")

conn.commit()
# conn.close()

In [5]:
sqlq("SELECT name FROM sqlite_master Where type='table'", conn)

,name


In [6]:
cursor.execute(''' 
CREATE TABLE IF NOT EXISTS proveedores (
    proveedor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    direccion TEXT,
    ciudad TEXT,
    provincia TEXT,
    codigo_proveedor TEXT UNIQUE NOT NULL
)
''')

In [7]:
prov_data = [
    ('Proveedor A', 'Calle Madrid', 'Madrid', 'Madrid', 'PROV1'),
    ('Proveedor B', 'Calle Barna', 'Barcelona', 'Barcelona', 'PROV2'),
    ('Proveedor C', 'Calle San Pío X', 'Valencia', 'Valencia', 'PROV3'),
    ('Proveedor D', 'Calle Estafeta', 'Pamplona', 'Navarra', 'PROV4'),
    ('Proveedor E', 'Calle Laurel', 'Logroño', 'Logroño', 'PROV5')
]
cursor.executemany(''' 
INSERT INTO proveedores (nombre, direccion, ciudad, provincia, codigo_proveedor)
VALUES (?,?,?,?,?)
''', prov_data)

In [8]:
sqlq("SELECT * FROM proveedores", conn)

,proveedor_id,nombre,direccion,ciudad,provincia,codigo_proveedor
0,1,Proveedor A,Calle Madrid,Madrid,Madrid,PROV1
1,2,Proveedor B,Calle Barna,Barcelona,Barcelona,PROV2


In [9]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS categorias (
    categoria_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    codigo_categoria TEXT UNIQUE NOT NULL
)
''')

In [10]:
cat_data = [
    ('Vino', '001'),
    ('Distillado', '002')
]
cursor.executemany('''
INSERT INTO categorias (nombre, codigo_categoria)
VALUES (?,?)
''', cat_data)

In [11]:
sqlq("SELECT * FROM categorias", conn)

,categoria_id,nombre,codigo_categoria
0,1,Vino,001
1,2,Distillado,002


In [12]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS piezas (
    pieza_id INTEGER PRIMARY KEY AUTOINCREMENT,
    codigo_pieza TEXT UNIQUE NOT NULL,
    nombre TEXT NOT NULL,
    color TEXT,
    precio FLOAT,
    categoria_id INTEGER,
    FOREIGN KEY (categoria_id) REFERENCES categorias(categoria_id)
)
''')

In [13]:
pieza_data = [
    ('P1', 'Sotovelo', 'Blanco', 20.50, '001'),
    ('P2', '10 Mil Botellas', 'Ambar', 80.70, '002'),
    ('P2', 'Domus Aurea', 'Tinto', 15.50, '001'),
    ('P2', 'Mar de Frades', 'Blanco', 20.50, '002'),
    ('P2', 'Ron Havana Selección de Maestros', 'Ambar', 65.50, '001')

]
cursor.executemany(''' 
INSERT INTO piezas (codigo_pieza, nombre, color, precio, categoria_id)
VALUES (?,?,?,?,?)
''', pieza_data)

In [14]:
sqlq("SELECT * FROM piezas", conn)

,pieza_id,codigo_pieza,nombre,color,precio,categoria_id
0,1,P1,Sotovelo,Blanco,20.5,1
1,2,P2,10 Mil Botellas,Ambar,80.7,2


In [21]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS suministros (
    suminstros_id INTEGER PRIMARY KEY AUTOINCREMENT,
    proveedor_id INTEGER,
    pieza_id INTEGER,
    fecha DATETIME DEFAULT ON UPDATE,
    cantidad INTEGER NOT NULL,
    FOREIGN KEY (proveedor_id) REFERENCES proveedores(proveedor_id)
    FOREIGN KEY (pieza_id) REFERENCES piezas(pieza_id) 
)
''')

OperationalError: near "ON": syntax error

In [ ]:
sumin_data = [
    (1, 1, '2024-07-01', 120),
    (2, 2, '2024-07-02', 30)
]
cursor.executemany('''
INSERT INTO suministros (proveedor_id, pieza_id, fecha, cantidad)
VALUES (?,?,?,?)
''', sumin_data)

In [ ]:
sqlq("SELECT * FROM suministros", conn)

,suminstros_id,proveedor_id,pieza_id,fecha,cantidad
0,1,1,1,2024-07-01,120
1,2,2,2,2024-07-02,30


In [16]:
sqlq('SELECT * FROM sqlite_master WHERE type == "table"', conn)

,type,name,tbl_name,rootpage,sql
0,table,proveedores,proveedores,2,CREATE TABLE proveedores (\n proveedor_id I...
1,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,categorias,categorias,5,CREATE TABLE categorias (\n categoria_id IN...
3,table,piezas,piezas,7,CREATE TABLE piezas (\n pieza_id INTEGER PR...


In [18]:
df_prov = sqlq('SELECT * FROM proveedores', conn)
df_prov

,proveedor_id,nombre,direccion,ciudad,provincia,codigo_proveedor
0,1,Proveedor A,Calle Madrid,Madrid,Madrid,PROV1
1,2,Proveedor B,Calle Barna,Barcelona,Barcelona,PROV2


In [19]:
df_categ = sqlq('SELECT * FROM categorias', conn)
df_categ

,categoria_id,nombre,codigo_categoria
0,1,Vino,001
1,2,Distillado,002


In [20]:
df_piezas = sqlq('SELECT * FROM piezas', conn)
df_piezas

,pieza_id,codigo_pieza,nombre,color,precio,categoria_id
0,1,P1,Sotovelo,Blanco,20.5,1
1,2,P2,10 Mil Botellas,Ambar,80.7,2


In [ ]:
conn.commit()
conn.close()